In [3]:
from ecoli.experiments.ecoli_master_sim import EcoliSim
import numpy as npy
import os
import json 
import sys 
import copy

DEFAULT_ECOLISIM_CONFIG_PATH = "../resources/config/default.json"
DEFAULT_DURATION = 1111
DEFAULT_INTERVAL = 1.0


def compile_simulation(
    config_fp: str | None = None, 
    experiment_id: str | None = None,
    duration: float | None = None, 
    time_step: float | None = None,
    framesize: float | None = None,
    datadir: str | None = None,
):
    # load config file TODO: replace with pbg doc!
    composite_config_fp = config_fp or DEFAULT_ECOLISIM_CONFIG_PATH
    with open(composite_config_fp, 'r') as f:
        config = copy.deepcopy(json.load(f))
    if experiment_id is not None:
        config['experiment_id'] = experiment_id
    print('Running sim')
    # override save dir
    if datadir is not None:
        config['daughter_outdir'] = datadir

    # configure times and savetimes
    dur = duration or config.get('total_time', DEFAULT_DURATION)
    timestep = time_step or config.get('time_step', DEFAULT_INTERVAL)
    t = npy.arange(1, dur, framesize or timestep)
    config['save'] = True
    config['save_times'] = t.tolist()
    # build, compile, and run sim
    # if datadir is not None and not os.path.exists(datadir):
    #     os.mkdir(datadir)
    
    sim = EcoliSim(config, datadir)
    sim.total_time = dur
    sim.time_step = timestep
    sim.build_ecoli()
    return sim


def dispatch_simulation(
    config_fp: str | None = None, 
    experiment_id: str | None = None,
    duration: float | None = None, 
    time_step: float | None = None,
    framesize: float | None = None,
    datadir: str | None = None,
):
    # load config file TODO: replace with pbg doc!
    composite_config_fp = config_fp or DEFAULT_ECOLISIM_CONFIG_PATH
    with open(composite_config_fp, 'r') as f:
        config = copy.deepcopy(json.load(f))
    if experiment_id is not None:
        config['experiment_id'] = experiment_id
    print('Running sim')
    # override save dir
    if datadir is not None:
        config['daughter_outdir'] = datadir

    # configure times and savetimes
    dur = duration or config.get('total_time', DEFAULT_DURATION)
    timestep = time_step or config.get('time_step', DEFAULT_INTERVAL)
    t = npy.arange(1, dur, framesize or timestep)
    config['save'] = True
    config['save_times'] = t.tolist()
    # build, compile, and run sim
    # if datadir is not None and not os.path.exists(datadir):
    #     os.mkdir(datadir)
    
    sim = EcoliSim(config, datadir)
    sim.build_ecoli()
    sim.total_time = dur
    sim.time_step = time_step

    return sim.run()

In [4]:
duration = 111.0
time_step = 1.0
framesize = time_step
experiment_id="notebook0"
datadir=f'./data/{experiment_id}'

sim = compile_simulation(duration=duration, time_step=time_step, framesize=framesize, datadir=datadir)

Running sim


In [5]:
dir(sim)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_retrieve_process_configs',
 '_retrieve_processes',
 '_retrieve_topology',
 'add_processes',
 'agent_id',
 'amp_lysis',
 'build_ecoli',
 'chromosome_path',
 'condition',
 'config',
 'd_period',
 'data_dir',
 'daughter_outdir',
 'description',
 'divide',
 'division_threshold',
 'division_variable',
 'ecoli',
 'ecoli_experiment',
 'emit_config',
 'emit_paths',
 'emit_processes',
 'emit_topology',
 'emit_unique',
 'emitter',
 'emitter_arg',
 'engine_process_reports',
 'exclude_processes',
 'experiment_id',
 'experiment_id_base',
 'export_json',
 'fail_at_total_time',
 'fixed_media',
 'flow',
 'from_cli',
 '

In [10]:
type(sim.ecoli)

vivarium.core.composer.Composite

In [13]:
sim.ecoli.get_parameters()

{'agents': {'0': {'bulk-timeline': {'time_step': 1.0,
    'timeline': {0: {('media_id',): 'minimal'}},
    'timestep': 1.0},
   'global_clock': {'timestep': 1.0}}}}

In [15]:
def f(x: 11.11):
    print(x)

f(2)

2


In [13]:
from IPython.display import display, HTML

ws_url = "ws://localhost:8080/"

html = """
<div id="ask">
  <input placeholder="Enter your name" />
  <a href="#">Join</a>
</div>

<div id="channel" style="display: none;">
  <input id="message" placeholder="Type a message" />
  <ul id="msgs"></ul>
  <button id="wsSendBtn">Send Message</button>
</div>

<script>
(function() {{
  if (!window.WebSocket) {{
    alert("WebSocket not supported in this browser.");
    return;
  }}

  const askDiv = document.getElementById("ask");
  const input = askDiv.querySelector("input");
  const joinBtn = askDiv.querySelector("a");

  const channel = document.getElementById("channel");
  const msgInput = document.getElementById("message");
  const msgList = document.getElementById("msgs");
  const button = document.getElementById('wsSendBtn');

  let ws;

  function join(name) {{
    ws = new WebSocket('ws://localhost:8080/');

    ws.onmessage = function(evt) {{
      const obj = JSON.parse(evt.data);
      const li = document.createElement("li");
      li.textContent = obj;
      msgList.appendChild(li);
    }};

    ws.onopen = function() {{
      const li = document.createElement("li");
      li.textContent = "Connected as " + name;
      msgList.appendChild(li);
    }};

    ws.onerror = function(err) {{
      const li = document.createElement("li");
      li.textContent = "Error: " + err;
      msgList.appendChild(li);
    }};
  }}

  joinBtn.onclick = function() {{
    const name = input.value || "anonymous";
    join(name);
    askDiv.style.display = "none";
    channel.style.display = "block";
    msgInput.focus();
  }};

  input.addEventListener("keydown", function(e) {{
    if (e.key === "Enter") {{
      joinBtn.click();
    }}
  }});
  button.onclick = function() {
      const msg = msgInput.value;
      if (ws.readyState === WebSocket.OPEN) {
        ws.send(msg);
        log("📤 Sent: " + msg);
      } else {
        log("⚠️ WebSocket not ready.");
      }
    };
}})();
</script>
"""

display(HTML(html))
